In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import glob
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.externals import joblib
from skimage.transform import resize
from tqdm import tqdm_notebook as tqdm
import ipywidgets as ipy

from common_blocks.utils import plot_list, read_images
from common_blocks.metrics import compute_ious, compute_eval_metric

METADATA_FILEPATH = 'YOUR/metadata.csv'
METADATA_FILEPATH = '/mnt/ml-team/minerva/open-solutions/salt/files/metadata.csv'

OUT_OF_FOLD_TRAIN_RESULTS_FILEPATH = 'YOUR/validation_results.pkl'
OUT_OF_FOLD_TRAIN_RESULTS_FILEPATH = '/mnt/ml-team/minerva/open-solutions/salt/kuba/experiments/sal_907_cv_812_lb_820/out_of_fold_train_predictions.pkl'

In [ ]:
def load_img(path):
    img = np.array(Image.open(path))
    return img

In [ ]:
metadata = pd.read_csv(METADATA_FILEPATH)

oof_train = joblib.load(OUT_OF_FOLD_TRAIN_RESULTS_FILEPATH)
ids = oof_train['ids']#[:100]
predictions = oof_train['images']#[:100]

In [ ]:
THRESHOLD = 0.5

predicted_maps, predicted_masks, masks, images, depths, sizes = [],[],[],[],[],[]
for idx, pred in tqdm(zip(ids, predictions)):
    row = metadata[metadata['id']==idx]
    predicted_map = resize(pred[1,:,:],(101,101),mode='constant')
    predicted_mask = (predicted_map > THRESHOLD).astype(int)
    image = load_img(row.file_path_image.values[0])
    mask = (load_img(row.file_path_mask.values[0]) > 0).astype(int)
    depth = row.z.values[0]
    size = np.sum(mask)
    
    images.append(image)
    masks.append(mask)
    depths.append(depth)
    predicted_maps.append(predicted_map)
    predicted_masks.append(predicted_mask)
    sizes.append(size)

In [ ]:
@ipy.interact(idx = ipy.IntSlider(min=0,max=4000,value=0,step=1))
def present(idx=idx):
    predicted_map = predicted_maps[idx]
    predicted_mask = predicted_masks[idx]
    image=images[idx]
    mask=masks[idx]
    size = sizes[idx]
    iout = compute_eval_metric(mask, predicted_mask)
    print('IOUT {} size {} depth {}'.format(iout, size, depth))
    plot_list(images=[image,predicted_map],labels=[predicted_mask, mask])

# Problems

1. Symmetric predictions near boundaries (reflection padding ?)

 **idx**: 3, 10, 26, 41, 42, 43, 45, 47, 48, 122, 125, 126, 128, 137
 
2. Unfilled stuff near boundaries

 **idx**: 2, 4, 23, 27
 
3. Disconnected components

 **idx**: 3
 
4. Weird round predictions

 **idx**: 6
 
5. Unexplained stuff close to boundaries

 **idx** 32

# This will take a while on the entire dataset it is probably better to take a sample

In [ ]:
iouts = [compute_eval_metric(gt, pred) for gt, pred in tqdm(zip(masks, predicted_masks))]
results = list(zip(iouts, sizes, depths, images, predicted_masks, predicted_maps, masks))

# Score distributions

In [ ]:
sns.distplot(iouts)

It seems that the model is either really good or really bad.
For example:

In [ ]:
print(np.mean([score for score in iouts if score>0.1]))

# score by depth

In [ ]:
sns.regplot(depths, iouts, fit_reg=False)

# score by size

In [ ]:
sns.regplot(sizes, iouts, fit_reg=False)

It seems that the model is better for lower depths.
Not sure yet what to do with it.

# Predicted mask exploration

In [ ]:
def filter_iout(results, iout_range):
    iout_min, iout_max = iout_range
    results_filtered = []
    for tup in results:
        if iout_min<=tup[0]<=iout_max:
            results_filtered.append(tup)
    return results_filtered

def filter_size(results, size_range):
    size_min, size_max = size_range
    results_filtered = []
    for tup in results:
        if size_min<=tup[1]<=size_max:
            results_filtered.append(tup)
    return results_filtered

In [ ]:
IMG_NR = 10
# results_filtered = filter_iout(results, iout_range=(0.0,0.2))
results_filtered = filter_size(results, size_range=(1000, 100000))
print(len(results_filtered))

for iout, s, z, img, pred_mask, pred_map, gt in results_filtered[:IMG_NR]:
    print('IOUT {}, size {}, depth {}'.format(iout, s, z))
    plot_list(images=[img, pred_map],labels=[pred_mask, gt])